In [36]:
from bs4 import BeautifulSoup
import re
from threading import Thread
import urllib
import pandas as pd
import urllib.request

In [37]:
#List of yelp urls to scrape
#function that will do actual scraping job
#Need to resolve checking of availability of data!!
def scrape(ur):
    with urllib.request.urlopen(ur) as url:
        html = url.read()
    soup = BeautifulSoup(html,"html.parser")

    retaurant_name = soup.find('h1')
    business_info = {}
    business_info['restaurant_name']= str(retaurant_name.text.strip().rstrip())
    for i in soup.find('div',{'class':'short-def-list'}).findAll('dl'):
        key = i.find('dt').text.strip().rstrip()
        value = i.find('dd').text.strip().rstrip()
        business_info[str(key)]=str(value)
    retaurant_address = soup.find('span',itemprop="streetAddress")
    business_info['retaurant_address'] = str(retaurant_address.text.strip().rstrip())
    restaurant_zipcode = soup.find('span',itemprop="postalCode")
    business_info['restaurant_zipcode'] = str(restaurant_zipcode.text.strip().rstrip())
    restaurant_reviewcount = soup.find('span',itemprop="reviewCount")
    business_info['restaurant_reviewcount'] = str(restaurant_reviewcount.text.strip().rstrip())
    #rating = soup.find(itemprop="ratingValue").get("content")
    #business_info['restaurant_rating'] = str(rating.text.strip().rstrip())
    return business_info

In [38]:
def get_urls_from_search(term, location, num):
    
    term = term.replace(' ','+')
    location = location.replace(' ','+')
    query = 'https://www.yelp.com/search?find_desc='+term+'&find_loc='+location+'&start='+str(num*10)
    with urllib.request.urlopen(query) as url:
        contents = url.read()
    #contents = urllib.urlopen(query).read()
    soup = BeautifulSoup(contents, "html.parser")
    #print(soup)
    business_url = []
    for result in soup.findAll('a',{'class':'biz-name js-analytics-click'}):
        business_url.append("http://www.yelp.com" + result['href'])
    return business_url

In [5]:
# List of all locations
# 'Alphabet_City','Battery_Park','Chelsea','Chinatown','Civic_Center','East_Harlem','East_Village','Financial_District','Flatiron','Gramercy','Greenwich_Village','Harlem','Hell\'s_Kitchen','Inwood','Kips_Bay','Koreatown','Little_Italy','Lower_East_Side','Manhattan_Valley','Marble_Hill','Meatpacking_District','Midtown_East','Midtown_West','Morningside_Heights','Murray_Hill','NoHo','Nolita','Roosevelt_Island','SoHo','South_Street_Seaport','South_Village','Stuyvesant_Town','Theater_District','TriBeCa','Two_Bridges','Union_Square','Upper_East_Side','Upper_West_Side','Washington_Heights','West_Village', 'Yorkville'

# Yorkville entirely encompassed by Upper East Side
# Theater District entirely encompassed by Midtown West

#searchLocations = ['Alphabet_City','Battery_Park','Chelsea','Chinatown','Civic_Center','East_Harlem','East_Village','Financial_District','Flatiron','Gramercy','Greenwich_Village','Harlem','Hell\'s_Kitchen','Inwood','Kips_Bay','Koreatown','Little_Italy','Lower_East_Side','Manhattan_Valley','Marble_Hill','Meatpacking_District','Midtown_East','Midtown_West','Morningside_Heights','Murray_Hill','NoHo','Nolita','Roosevelt_Island','SoHo','South_Street_Seaport','South_Village','Stuyvesant_Town','TriBeCa','Two_Bridges','Union_Square','Upper_East_Side','Upper_West_Side','Washington_Heights','West_Village']

searchLocations = ['Battery_Park','Chelsea','Chinatown'] #test
max_num = 50 
# 500 

In [6]:
max_num = 10
urls_set = []
for i, loc in enumerate(searchLocations):
    # now run for loop with fix location and food type and append urls 
    # page ranking is based on relevance ranked by yelp
    for num in range(0,max_num):
        urls = get_urls_from_search('Japanese Food',loc, num)
        urls = urls[1:] # 0th link is irrelavant
        # len(urls)=0 if the starting page number exceed the maximum possible
        if (len(urls) ==0):
            break
        else:
            for i in range(0,len(urls)-1):
                urls_set.append(urls[i])
    # convert the urls from list to csv file for each location
    pd_urls_set = pd.DataFrame(urls_set)
    pd_urls_set
    pd_urls_set.to_csv("urls_set_{0}.csv".format(loc))

In [39]:
urls = get_urls_from_search('Japanese Food','New York',3)
urls = urls[1:]

In [40]:
info = {} # create a set
for u in urls:
    print(u)
    url_dict=scrape(u)
    for key,value in url_dict.items():
        info.setdefault(key,[]).append(value)

http://www.yelp.com/biz/sakamai-new-york?osq=Japanese+Food
http://www.yelp.com/biz/ootoya-times-square-new-york-2?osq=Japanese+Food
http://www.yelp.com/biz/sushi-daizen-long-island-city?osq=Japanese+Food
http://www.yelp.com/biz/1-or-8-brooklyn?osq=Japanese+Food
http://www.yelp.com/biz/tonkatsu-matsunoya-new-york?osq=Japanese+Food
http://www.yelp.com/biz/momokawa-new-york-2?osq=Japanese+Food
http://www.yelp.com/biz/o-ya-new-york?osq=Japanese+Food
http://www.yelp.com/biz/mito-asian-fusion-forest-hills?osq=Japanese+Food
http://www.yelp.com/biz/okuda-new-york?osq=Japanese+Food
http://www.yelp.com/biz/fukurou-new-york?osq=Japanese+Food


In [41]:
header=info.keys()
info={}
for u in urls:
    url_dict=scrape(u)
    for i in header:
        if i in url_dict.keys():
            info.setdefault(i,[]).append(url_dict[i])
        else:
            info.setdefault(i,[]).append('NA')
    

In [42]:
final_df = pd.DataFrame(info)
final_df

,Accepts Android Pay,Accepts Apple Pay,Accepts Bitcoin,Accepts Credit Cards,Alcohol,Ambience,Attire,Bike Parking,Caters,Delivery,...,Parking,Take-out,Takes Reservations,Waiter Service,Wheelchair Accessible,Wi-Fi,restaurant_name,restaurant_reviewcount,restaurant_zipcode,retaurant_address
0,No,Yes,No,Yes,Full Bar,"Intimate, Classy",Casual,Yes,Yes,Yes,...,Street,No,Yes,NA,NA,No,SakaMai,518,10002,157 Ludlow St
1,No,No,No,Yes,Full Bar,Casual,Casual,Yes,No,No,...,Street,No,Yes,NA,NA,No,Ootoya Times Square,678,10036,141 W 41st St
2,NA,No,No,Yes,Beer & Wine Only,Intimate,Casual,No,No,No,...,Street,No,Yes,Yes,No,Free,Sushi Daizen,78,11101,47-38 Vernon Blvd
3,NA,No,NA,Yes,Full Bar,Trendy,Casual,Yes,No,No,...,Street,No,Yes,NA,NA,No,1 or 8,373,11249,66 S 2nd St
4,No,No,No,Yes,Beer & Wine Only,Casual,Casual,No,NA,Yes,...,Street,Yes,Yes,Yes,NA,No,Tonkatsu Matsunoya,219,10017,131 E 45th St
5,No,No,No,Yes,Beer & Wine Only,Casual,Casual,Yes,Yes,Yes,...,Street,Yes,Yes,Yes,NA,No,Momokawa,68,10075,1466 1st Ave
6,No,No,NA,Yes,Beer & Wine Only,Trendy,Casual,No,No,No,...,NA,No,Yes,Yes,NA,No,O Ya,184,10016,120 E 28th St
7,Yes,Yes,NA,Yes,Full Bar,"Casual, Trendy, Romantic, Classy",Casual,Yes,Yes,Yes,...,Street,Yes,Yes,Yes,Yes,Free,Mito Asian Fusion,212,11375,64-18 108th St
8,NA,NA,NA,Yes,NA,NA,NA,NA,NA,NA,...,NA,NA,Yes,NA,NA,NA,奥田,7,10011,458 W 17th St
9,NA,No,NA,Yes,Beer & Wine Only,Casual,Casual,Yes,No,No,...,Street,No,Yes,NA,NA,No,Fukurou,262,10012,87 MacDougal St


In [43]:
final_df.to_csv("final_df.csv")

In [44]:
dict = scrape('https://www.yelp.com/biz/mount-everest-indias-cuisine-las-vegas?osq=indian+food')
print(dict)

{'restaurant_name': 'Mount Everest India’s Cuisine', 'Takes Reservations': 'Yes', 'Delivery': 'No', 'Take-out': 'Yes', 'Accepts Credit Cards': 'Yes', 'Accepts Apple Pay': 'No', 'Accepts Android Pay': 'No', 'Good For': 'Lunch, Dinner', 'Parking': 'Private Lot', 'Bike Parking': 'Yes', 'Wheelchair Accessible': 'Yes', 'Good for Kids': 'Yes', 'Good for Groups': 'Yes', 'Attire': 'Casual', 'Ambience': 'Casual', 'Noise Level': 'Average', 'Alcohol': 'Beer & Wine Only', 'Outdoor Seating': 'No', 'Wi-Fi': 'No', 'Has TV': 'Yes', 'Waiter Service': 'Yes', 'Caters': 'Yes', 'retaurant_address': '3641 W Sahara Ave', 'restaurant_zipcode': '89102', 'restaurant_reviewcount': '1239'}
